#**FELLOW 6**
Ensemble Competition

###**Import Dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_train = pd.read_csv('/content/gdrive/MyDrive/Fellow6/Train.csv')
df_test = pd.read_csv('/content/gdrive/MyDrive/Fellow6/Test.csv')

In [ ]:
df_train.head()

,date,day,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Thursday,0.65,23.69,861.0,7200,0,0.0,0,0,60.0,0.521180
1,1/3/2015,Saturday,0.70,23.69,544.0,13800,0,0.0,0,0,60.0,0.700134
2,1/4/2015,Sunday,0.80,28.08,878.0,4260,50,0.0,0,0,55.5,0.800598
3,1/5/2015,Monday,0.80,28.08,759.0,6900,50,0.0,0,0,57.5,0.800077
4,1/6/2015,Tuesday,0.80,28.08,925.0,10350,50,0.0,0,0,57.5,0.800594


In [ ]:
df_train.shape

(48, 12)

In [ ]:
df_train.dtypes

date                      object
day                       object
targeted_productivity    float64
smv                      float64
wip                      float64
over_time                  int64
incentive                  int64
idle_time                float64
idle_men                   int64
no_of_style_change         int64
no_of_workers            float64
actual_productivity      float64
dtype: object

### **One Hot Encoding for Variable Day**

In [ ]:
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

encoding = OneHotEncoder()
encoding.fit(df_train[['day']])
encoding_df=encoding.transform(df_train[['day']]).toarray()

In [ ]:
X = np.concatenate([encoding_df,np.array(df_train[['targeted_productivity',	'smv',	'wip',	'over_time',	'incentive',	'idle_time',	'idle_men'	,'no_of_style_change',	'no_of_workers']])],axis = 1)
y = df_train['actual_productivity']

###**DATA SPLITTING**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test1, y_train, y_test1 = train_test_split(X, y, test_size=0.2, random_state=42)

###**MODELLING**

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor(learning_rate = 0.04, n_estimators = 2500, silent =  1, max_depth = 7, booster = 'dart', objective = 'reg:tweedie', random_state=42)
my_model.fit(X_train, y_train,verbose=False)

XGBRegressor(base_score=0.5, booster='dart', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.04, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None, n_estimators=2500,
             n_jobs=1, nthread=None, objective='reg:tweedie', random_state=42,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None, silent=1,
             subsample=1, verbosity=1)

### **Calculate MAPE (Mean Absolut Percentage Error)**


In [ ]:
predictions = my_model.predict(X_test1)

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

mape(y_test1, predictions)

3.9304467570388586

In [ ]:
from sklearn import metrics

r2 = metrics.r2_score(y_test1, predictions)
print('R2: {}\n'.format(r2))

mse = metrics.mean_squared_error(y_test1, predictions)
print('MSE: {}\n'.format(mse))

print('RMSE: {}\n'.format(np.sqrt(mse)))

mae = metrics.mean_absolute_error(y_test1, predictions)
print('MAE: {}\n'.format(mae))

R2: 0.6421187567390807

MSE: 0.0021171037884445713

RMSE: 0.046011996136274845

MAE: 0.028466053063574238



In [ ]:
# Fixed

from xgboost import XGBRegressor

my_model = XGBRegressor(learning_rate = 0.04, n_estimators = 1500000, silent =  1, max_depth = 7)
my_model.fit(X_train, y_train,verbose=False)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.04, max_delta_step=0,
             max_depth=7, min_child_weight=1, missing=None,
             n_estimators=1500000, n_jobs=1, nthread=None,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, seed=None, silent=1, subsample=1, verbosity=1)

In [ ]:
predictions = my_model.predict(X_test1)

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

mape(y_test1, predictions)

3.3072358294570092

In [ ]:
from sklearn import metrics

r2 = metrics.r2_score(y_test1, predictions)
print('R2: {}\n'.format(r2))

mse = metrics.mean_squared_error(y_test1, predictions)
print('MSE: {}\n'.format(mse))

print('RMSE: {}\n'.format(np.sqrt(mse)))

mae = metrics.mean_absolute_error(y_test1, predictions)
print('MAE: {}\n'.format(mae))

R2: 0.7327454410381888

MSE: 0.0015809871288633686

RMSE: 0.03976162885073207

MAE: 0.023834411126699838



### **DATA PREDICTION**

In [ ]:
encoding_test = encoding.transform(df_test[['day']]).toarray()
X_test = np.concatenate([encoding_test,np.array(df_test[['targeted_productivity',	'smv',	'wip',	'over_time',	'incentive',	'idle_time',	'idle_men'	,'no_of_style_change',	'no_of_workers']])],axis = 1)

In [ ]:
xgb_predict = my_model.predict(X_test)
print(xgb_predict)

[0.51897824 0.5235987  0.6158876  0.6611476  0.75       0.75
 0.74264216 0.75       0.8285024  0.8625996  0.8430569 ]


In [ ]:
df_xgb_predict = pd.DataFrame(xgb_predict)
df_xgb_predict

,0
0,0.518978
1,0.523599
2,0.615888
3,0.661148
4,0.750000
5,0.750000
6,0.742642
7,0.750000
8,0.828502
9,0.862600


In [ ]:
df_xgb_predict.index+=1
df_xgb_predict=df_xgb_predict.rename(columns = {0: 'actual_productivity'}, inplace = False)
df_xgb_predict

,actual_productivity
1,0.518978
2,0.523599
3,0.615888
4,0.661148
5,0.750000
6,0.750000
7,0.742642
8,0.750000
9,0.828502
10,0.862600


In [ ]:
#Rename index into "id"

df_xgb_predict = df_xgb_predict.rename_axis('id').reset_index()
print(df_xgb_predict)

    id  actual_productivity
0    1             0.518978
1    2             0.523599
2    3             0.615888
3    4             0.661148
4    5             0.750000
5    6             0.750000
6    7             0.742642
7    8             0.750000
8    9             0.828502
9   10             0.862600
10  11             0.843057


### **EXPORT TO CSV**

In [ ]:
df_xgb_predict.to_csv('XGB_Prediction2.csv', index=False)